<a href="https://colab.research.google.com/github/HesusG/diagnostico-lineas-accion/blob/main/Semana2/notebooks/01_ji_cuadrada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba Ji-Cuadrada (χ²) de Independencia

## Objetivos
- Entender cuándo usar la prueba Ji-cuadrada
- Crear y analizar tablas de contingencia
- Interpretar el estadístico χ² y p-value
- Analizar residuos estandarizados
- Calcular medidas de asociación (Cramér's V)

---

## 1. Preparación

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, chi2
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
%matplotlib inline

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/HesusG/diagnostico-lineas-accion/main/Semana1/datos/ejemplo_satisfaccion_clientes.csv')
print(f"Dataset cargado: {df.shape[0]} registros")
df.head()

## 2. Recordatorio: ¿Cuándo usar Ji-Cuadrada?

✅ **Usar cuando:**
- Ambas variables son **categóricas**
- Quieres probar si hay **asociación/relación** entre ellas
- Tienes datos de frecuencias o conteos

**Pregunta de investigación:**
> ¿El área de servicio está relacionada con el nivel de satisfacción?

---

## 3. Preparación de Datos: Categorizar Variables

Primero, convertimos satisfacción (numérica) en categórica:

In [ ]:
# Crear variable categórica de satisfacción
# Baja: 1-6, Media: 7-8, Alta: 9-10
df['satisfaccion_cat'] = pd.cut(df['satisfaccion'],
                                  bins=[0, 6, 8, 10],
                                  labels=['Baja', 'Media', 'Alta'])

print("Distribución de satisfacción categorizada:")
print(df['satisfaccion_cat'].value_counts().sort_index())

# Verificar que no haya valores nulos
print(f"\nValores nulos: {df['satisfaccion_cat'].isnull().sum()}")

## 4. Tabla de Contingencia

Una **tabla de contingencia** muestra las frecuencias de las combinaciones de dos variables categóricas.

In [ ]:
# Crear tabla de contingencia
tabla = pd.crosstab(df['area'], df['satisfaccion_cat'])

print("="*60)
print("TABLA DE CONTINGENCIA: Área vs Satisfacción")
print("="*60)
print(tabla)
print("\nTotal de observaciones:", tabla.sum().sum())

In [ ]:
# Tabla con totales marginales
tabla_con_totales = pd.crosstab(df['area'], df['satisfaccion_cat'], margins=True, margins_name='TOTAL')
print("\nTabla con Totales Marginales:")
print(tabla_con_totales)

## 5. Visualización de la Tabla de Contingencia

In [ ]:
# Heatmap de frecuencias observadas
plt.figure(figsize=(10, 6))
sns.heatmap(tabla, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Frecuencia'})
plt.title('Frecuencias Observadas: Área vs Satisfacción', fontsize=14, fontweight='bold')
plt.xlabel('Nivel de Satisfacción', fontsize=12)
plt.ylabel('Área', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de barras apiladas
tabla.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis', edgecolor='black')
plt.title('Distribución de Satisfacción por Área', fontsize=14, fontweight='bold')
plt.xlabel('Área', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.legend(title='Satisfacción', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de barras agrupadas (alternativa)
tabla.plot(kind='bar', figsize=(12, 6), colormap='Set2', edgecolor='black')
plt.title('Comparación de Satisfacción entre Áreas', fontsize=14, fontweight='bold')
plt.xlabel('Área', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.legend(title='Satisfacción')
plt.xticks(rotation=0)
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 6. Proporciones (Análisis Descriptivo)

In [ ]:
# Proporciones por fila (% dentro de cada área)
proporciones_fila = tabla.div(tabla.sum(axis=1), axis=0) * 100

print("Proporciones por Área (% de cada columna dentro de cada fila):")
print(proporciones_fila.round(2))

# Interpretación
print("\n💡 Interpretación:")
for area in proporciones_fila.index:
    categoria_max = proporciones_fila.loc[area].idxmax()
    porcentaje_max = proporciones_fila.loc[area].max()
    print(f"   • {area}: {porcentaje_max:.1f}% tienen satisfacción {categoria_max}")

In [ ]:
# Visualizar proporciones
proporciones_fila.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis', edgecolor='black')
plt.title('Distribución Porcentual de Satisfacción por Área', fontsize=14, fontweight='bold')
plt.xlabel('Área', fontsize=12)
plt.ylabel('Porcentaje (%)', fontsize=12)
plt.legend(title='Satisfacción', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.ylim(0, 100)
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 7. Prueba Ji-Cuadrada de Independencia

### Hipótesis:
- **H₀:** Área y Satisfacción son **independientes** (no hay relación)
- **H₁:** Área y Satisfacción **NO son independientes** (sí hay relación)

In [ ]:
# Realizar prueba Ji-cuadrada
chi2_stat, p_value, dof, expected = chi2_contingency(tabla)

print("="*60)
print("RESULTADOS DE LA PRUEBA JI-CUADRADA")
print("="*60)
print(f"Estadístico χ²: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Grados de libertad: {dof}")
print(f"Nivel de significancia (α): 0.05")
print("="*60)

In [ ]:
# Decisión
alpha = 0.05

print("\n📊 DECISIÓN:")
if p_value < alpha:
    print(f"   p-value ({p_value:.4f}) < α ({alpha})")
    print("   ✗ RECHAZAMOS H₀")
    print("\n💡 CONCLUSIÓN:")
    print("   Existe una RELACIÓN SIGNIFICATIVA entre el área de servicio y la satisfacción.")
    print("   El área donde se brinda el servicio SÍ influye en el nivel de satisfacción.")
else:
    print(f"   p-value ({p_value:.4f}) ≥ α ({alpha})")
    print("   ✓ NO RECHAZAMOS H₀")
    print("\n💡 CONCLUSIÓN:")
    print("   No hay evidencia suficiente de relación entre área y satisfacción.")
    print("   Las variables parecen ser independientes.")

## 8. Frecuencias Esperadas

Las **frecuencias esperadas** son las que esperaríamos si las variables fueran independientes.

In [ ]:
# Convertir a DataFrame para mejor visualización
tabla_esperada = pd.DataFrame(expected,
                               index=tabla.index,
                               columns=tabla.columns)

print("Frecuencias Esperadas (bajo H₀):")
print(tabla_esperada.round(2))

In [ ]:
# Comparación visual: Observadas vs Esperadas
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Frecuencias Observadas
sns.heatmap(tabla, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar_kws={'label': 'Frecuencia'})
axes[0].set_title('Frecuencias Observadas', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Satisfacción')
axes[0].set_ylabel('Área')

# Frecuencias Esperadas
sns.heatmap(tabla_esperada, annot=True, fmt='.1f', cmap='Oranges', ax=axes[1], cbar_kws={'label': 'Frecuencia'})
axes[1].set_title('Frecuencias Esperadas (H₀)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Satisfacción')
axes[1].set_ylabel('Área')

plt.tight_layout()
plt.show()

## 9. Verificación de Supuestos

In [ ]:
# Supuesto: Frecuencias esperadas ≥ 5 en al menos 80% de las celdas
celdas_total = expected.size
celdas_bajas = (expected < 5).sum()
porcentaje_bajas = (celdas_bajas / celdas_total) * 100

print("="*60)
print("VERIFICACIÓN DE SUPUESTOS")
print("="*60)
print(f"Total de celdas: {celdas_total}")
print(f"Celdas con frecuencia esperada < 5: {celdas_bajas}")
print(f"Porcentaje: {porcentaje_bajas:.1f}%")

if porcentaje_bajas > 20:
    print("\n⚠️ ADVERTENCIA: Más del 20% de celdas tienen frecuencia esperada < 5")
    print("   Considerar:")
    print("   1. Combinar categorías")
    print("   2. Usar Test Exacto de Fisher (para tablas 2x2)")
    print("   3. Aumentar tamaño muestral")
else:
    print("\n✓ Supuesto CUMPLIDO: La prueba Ji-cuadrada es válida")

## 10. Residuos Estandarizados

Los **residuos** indican qué celdas contribuyen más al estadístico χ².

In [ ]:
# Calcular residuos estandarizados
residuos = (tabla - tabla_esperada) / np.sqrt(tabla_esperada)

print("Residuos Estandarizados:")
print(residuos.round(2))

print("\n📊 Interpretación:")
print("   |residuo| > 2: Contribución significativa al χ²")
print("   |residuo| > 3: Contribución muy significativa")
print("   Positivo: Más casos observados de lo esperado")
print("   Negativo: Menos casos observados de lo esperado")

In [ ]:
# Visualizar residuos
plt.figure(figsize=(10, 6))
sns.heatmap(residuos, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            vmin=-3, vmax=3, cbar_kws={'label': 'Residuo Estandarizado'})
plt.title('Residuos Estandarizados', fontsize=14, fontweight='bold')
plt.xlabel('Nivel de Satisfacción', fontsize=12)
plt.ylabel('Área', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Identificar celdas con mayor contribución
print("\nCeldas con |residuo| > 2 (contribución significativa):")
for area in residuos.index:
    for nivel in residuos.columns:
        res = residuos.loc[area, nivel]
        if abs(res) > 2:
            direccion = "MÁS" if res > 0 else "MENOS"
            print(f"   • {area} - {nivel}: residuo = {res:.2f}")
            print(f"     → {direccion} casos de lo esperado")

## 11. Medida de Asociación: Cramér's V

In [ ]:
# Calcular Cramér's V (medida de fuerza de asociación)
n = tabla.sum().sum()
min_dim = min(tabla.shape[0] - 1, tabla.shape[1] - 1)
cramers_v = np.sqrt(chi2_stat / (n * min_dim))

print("="*60)
print("CRAMÉR'S V (Fuerza de Asociación)")
print("="*60)
print(f"V de Cramér: {cramers_v:.4f}")

# Interpretación
if cramers_v < 0.1:
    fuerza = "Asociación DÉBIL o NULA"
elif cramers_v < 0.3:
    fuerza = "Asociación MODERADA"
else:
    fuerza = "Asociación FUERTE"

print(f"Interpretación: {fuerza}")
print("\nEscala:")
print("   V < 0.1: Débil")
print("   0.1 ≤ V < 0.3: Moderada")
print("   V ≥ 0.3: Fuerte")

## 12. Segundo Ejemplo: Género vs Recomendación

In [ ]:
# Crear tabla de contingencia
tabla2 = pd.crosstab(df['genero'], df['recomendaria'])

print("="*60)
print("EJEMPLO 2: GÉNERO vs RECOMENDACIÓN")
print("="*60)
print("\nTabla de Contingencia:")
print(tabla2)

# Prueba Ji-cuadrada
chi2_2, p_2, dof_2, expected_2 = chi2_contingency(tabla2)

print(f"\nResultados:")
print(f"   χ² = {chi2_2:.4f}")
print(f"   p-value = {p_2:.4f}")
print(f"   grados de libertad = {dof_2}")

# Decisión
if p_2 < 0.05:
    print(f"\n✗ Rechazamos H₀ (p={p_2:.4f} < 0.05)")
    print("   Conclusión: Hay relación entre género y recomendación")
else:
    print(f"\n✓ No rechazamos H₀ (p={p_2:.4f} ≥ 0.05)")
    print("   Conclusión: No hay relación entre género y recomendación")

In [ ]:
# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap
sns.heatmap(tabla2, annot=True, fmt='d', cmap='Greens', ax=axes[0])
axes[0].set_title('Género vs Recomendación', fontweight='bold')
axes[0].set_xlabel('Recomendaría')
axes[0].set_ylabel('Género')

# Gráfico de barras
tabla2.plot(kind='bar', ax=axes[1], color=['salmon', 'lightgreen'], edgecolor='black')
axes[1].set_title('Distribución de Recomendación por Género', fontweight='bold')
axes[1].set_xlabel('Género')
axes[1].set_ylabel('Frecuencia')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].legend(title='Recomendaría')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 13. Reporte Ejecutivo

In [ ]:
print("="*70)
print("REPORTE EJECUTIVO - PRUEBA JI-CUADRADA")
print("="*70)

print("\n🔬 PRUEBA 1: Área vs Satisfacción")
print(f"   χ² = {chi2_stat:.4f}, p-value = {p_value:.4f}")
print(f"   V de Cramér = {cramers_v:.4f}")

if p_value < 0.05:
    print("   ✗ Existe relación significativa")
    print("\n   📊 Hallazgos Clave:")
    
    # Identificar área con mayor satisfacción alta
    prop_alta = proporciones_fila['Alta']
    mejor_area = prop_alta.idxmax()
    peor_area = prop_alta.idxmin()
    
    print(f"   • Área {mejor_area}: {prop_alta.max():.1f}% con satisfacción alta")
    print(f"   • Área {peor_area}: {prop_alta.min():.1f}% con satisfacción alta")
    print(f"\n   💡 Recomendación: Investigar mejores prácticas del Área {mejor_area}")
else:
    print("   ✓ No hay relación significativa")
    print("   La satisfacción es similar entre áreas")

print("\n🔬 PRUEBA 2: Género vs Recomendación")
print(f"   χ² = {chi2_2:.4f}, p-value = {p_2:.4f}")

if p_2 < 0.05:
    print("   ✗ Existe relación significativa")
else:
    print("   ✓ No hay relación significativa")
    print("   La tasa de recomendación es similar entre géneros")

print("\n" + "="*70)

## 14. Ejercicios Propuestos

### Ejercicio 1: Área vs Recomendación
¿Existe relación entre el área de servicio y si los beneficiarios recomendarían el servicio?

### Ejercicio 2: Grupos de Edad
Crea grupos de edad (Joven: <35, Adulto: 35-55, Mayor: >55) y prueba si hay relación con satisfacción.

### Ejercicio 3: Análisis de Residuos
Para cualquier combinación que encuentres significativa, analiza los residuos estandarizados e identifica qué celdas contribuyen más.

### Ejercicio 4: Combinaciones Múltiples
Explora otras combinaciones de variables categóricas:
- Tiempo de servicio (categorizado) vs Satisfacción
- Género vs Área


In [ ]:
# Tu código aquí


---

## Resumen

En este notebook aprendiste a:
- ✓ Crear y analizar tablas de contingencia
- ✓ Realizar la prueba Ji-cuadrada de independencia
- ✓ Interpretar el estadístico χ² y p-value
- ✓ Calcular y visualizar frecuencias esperadas
- ✓ Analizar residuos estandarizados para identificar celdas clave
- ✓ Calcular Cramér's V como medida de asociación
- ✓ Verificar supuestos de la prueba
- ✓ Interpretar resultados en contexto de negocio

**Siguiente notebook:** ANOVA (Análisis de Varianza)
